<a href="https://colab.research.google.com/github/Adrianft25/Curso-Big-Data-2023/blob/main/SpaceX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Etapa 1: Obtencion de datos**

En esta práctica obtendrás los datos para predecir si un falcon 9 aterizará con éxito o no. Los datos serán recogidos mediante la API de SpaceX y nos aseguraremos de que siguen un formato adecuado para las fases siguientes. El siguiente es un ejemplo de un aterrizaje exitoso: 

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing\_1.gif)


Aquí puedes ver varios ejemplos de aterrizajes fallidos:




![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


La gran mayoría de aterrizajes fallidos son intencionados para la realización de diversos controles. Estos aterrizajes se llevan a cabo la mayoria en el oceano, veremos estadísticas al respecto.

## Objetivos



En esta práctica realizaras una GET request a la API de SpaceX. Tambien realizarás cierto data wrangling básico y limpieza de datos.

***


## Importación de librerias.

Nos importamos las siguientes librerías necesarias para el desarrollo de la práctica. 

In [64]:
import requests
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#Las opciones anteriores aseguran que pandas muestre el df completo en jupyternotebooks

Realicemos ahora una petición GET a la API de SpaceX. La URL es la siguiente.


In [65]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

* Usando la librería requests usa el método GET y pasale como argumento URL anterior, guardala en una variable `response`  y realiza el print de `response.content`

In [66]:
#INSERTA AQUÍ TU CÓDIGO
#requests.TIPODEPETICIÓN(URL)
response = requests.get(spacex_url)


Si la petición ha sido realizada correctamente el resultado del print debería ser similar al siguiente:


```
b'[{"fairings":{"reused":false,"recovery_attempt":false,"recovered":false, ...
```
(Recomendamos cerrar el resultado del print después de revisarlo para que google collab no sufra.)

Nuestra variable `response` contiene una grandísima cantidad de información sobre los SpaceX pero vamos a intentar formatearla para poder ver de que se trata exactamente y que nos interesa.


### Parte 1: Obtén los datos mediante peticiones GET y crea un dataframe

Hemos observado que la variable `response` tiene forma de diccionario (como la mayoría de respuestas a peticiones GET). Una forma efectiva de formatear dichos datos es transformarlos en un JSON y posteriormente en un dataframe de python:


In [67]:
data=response.json()
data=pd.json_normalize(data)

*Utilizando el dataframe anterior muestra las 5 primeras filas*

In [ ]:
#INSERTA AQUÍ TU CÓDIGO
data.head(5)

Observemos que la mayoría de datos interesantes no se muestran de forma explícita sino que son IDs. Estos IDs nos permiten obtener mas información haciendo pediciones GET a diferentes endpoints de la API.
Para mas infrmación: https://docs.spacexdata.com/

Revisando la documentación de la API nos hemos dado cuenta de que no son necesarias todas las columnas, por lo que vamos a reducir nuestro dataframe a lo crucial:

In [ ]:
#Reescribe el dataframe para quedarnos solo con las columnas'rocket', 'payloads','success', 'launchpad', 'cores', 'flight_number', 'date_utc'
#INSERTA AQUÍ TU CÓDIGO:
data = data[["rocket", "payloads", "success", "launchpad", "cores", "flight_number", "date_utc"]]
data

In [70]:
# Algunos cohetes tienen mas de una carga 'payload' o 'core'. 
# Estos datos nos darán problemas en un futuro, entonces nos limitearemos a aquellos que solamente tienen un core o una carga (payload):
data = data[data['cores'].map(len)==1]
#INSERTA AQUI TU CÓDIGO para payloads
data = data[data["payloads"].map(len)==1]

data


,rocket,payloads,success,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,[5eb0e4b5b6c3bb0006eeb1e1],False,5e9e4502f5090995de566f86,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,[5eb0e4b6b6c3bb0006eeb1e2],False,5e9e4502f5090995de566f86,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",2,2007-03-21T01:10:00.000Z
3,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e5],True,5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e6],True,5e9e4502f5090995de566f86,"[{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5,2009-07-13T03:35:00.000Z
5,5e9d0d95eda69973a809d1ec,[5eb0e4b7b6c3bb0006eeb1e7],True,5e9e4501f509094ba4566f84,"[{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",6,2010-06-04T18:45:00.000Z
...,...,...,...,...,...,...,...
180,5e9d0d95eda69973a809d1ec,[631614d7ffc78f3b85670716],True,5e9e4502f509094188566f88,"[{'core': '61c1ef45a4a2462678cbf45d', 'flight': 2, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecb075134e7cd'}]",181,2022-08-28T02:22:00.000Z
181,5e9d0d95eda69973a809d1ec,[630f63bf18702d4844fb5391],True,5e9e4502f509092b78566f87,"[{'core': '5f57c54a0622a633027900a1', 'flight': 7, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}]",182,2022-08-31T05:40:00.000Z
184,5e9d0d95eda69973a809d1ec,[63161699ffc78f3b85670719],True,5e9e4501f509094ba4566f84,"[{'core': '60b800111f83cc1e59f16438', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",185,2022-09-17T01:05:00.000Z
185,5e9d0d95eda69973a809d1ec,[631616a7ffc78f3b8567071a],True,5e9e4501f509094ba4566f84,"[{'core': '627843d657b51b752c5c5a53', 'flight': 4, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",186,2022-09-24T23:30:00.000Z


In [71]:
# Otro problema que tenemos es que ahora tenemos listas de longitud 1. 
# Las listas de python no son especialmente compatibles con SQL por lo que lo solucionaremos sacando el valor de la lista:
data['cores'] = data['cores'].map(lambda x : x[0])
#INSERTA AQUI TU CÓDIGO para hacer lo mismo con la columna payloads
data['payloads'] = data['payloads'].map(lambda x : x[0])

In [72]:
# El formato de la fecha no es especialmete cómodo por lo que lo formatearemos:
data['date'] = pd.to_datetime(data['date_utc']).dt.date

No se te olvide ejecutar la celda anterior (1 sola vez)

In [73]:
data

,rocket,payloads,success,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,False,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,False,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,True,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,True,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,True,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z,2010-06-04
...,...,...,...,...,...,...,...,...
180,5e9d0d95eda69973a809d1ec,631614d7ffc78f3b85670716,True,5e9e4502f509094188566f88,"{'core': '61c1ef45a4a2462678cbf45d', 'flight': 2, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecb075134e7cd'}",181,2022-08-28T02:22:00.000Z,2022-08-28
181,5e9d0d95eda69973a809d1ec,630f63bf18702d4844fb5391,True,5e9e4502f509092b78566f87,"{'core': '5f57c54a0622a633027900a1', 'flight': 7, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}",182,2022-08-31T05:40:00.000Z,2022-08-31
184,5e9d0d95eda69973a809d1ec,63161699ffc78f3b85670719,True,5e9e4501f509094ba4566f84,"{'core': '60b800111f83cc1e59f16438', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",185,2022-09-17T01:05:00.000Z,2022-09-17
185,5e9d0d95eda69973a809d1ec,631616a7ffc78f3b8567071a,True,5e9e4501f509094ba4566f84,"{'core': '627843d657b51b752c5c5a53', 'flight': 4, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",186,2022-09-24T23:30:00.000Z,2022-09-24


-------------------

Si revisamos `rocket, payload,launchpad y cores` son IDs, estos IDs nos permiten hacer peticiones a diferetes endpoints y obtener más información. La información para cada variable se encuentra en los siguientes links:

*   Para <code>rocket</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/rockets/v4/one.md . De aquí nos quedaremos la versión del cohete propulsor.

*   Para <code>payload</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/payloads/v4/one.md . De aquí guardaremos la carga, la masa de esta carga, la orbita a la que fue enviada y el cliente.

*   Para <code>launchpad</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/launchpads/v4/one.md . Guardamos la latitud, longitud y nombre de las diferentes plataformas de lanzamiento.

*   Para <code>cores</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/cores/v4/one.md . Guardamos diferentes variables del nucleo.

Los datos serán guardados en listas y estas listas las utilizaremos para crear un nuevo dataframe con todos los datos:



In [74]:
#Para rocket
BoosterVersion = [] 

#Para paylaod
PayloadMass = []
Payload = []
Orbit = []
Customers= []

#Para launchpad
LaunchSite = []
Longitude = []
Latitude = []

#Para core
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
Mission_Outcome = []
ReusedCount = []
Serial = []


Para completar las listas anteriores nos definiremos una lista de funciones para facilitar la obtención de los datos.

Empecemos con los cohetes:

In [75]:
# DEFINICIÓN FUNCIÓN 1
def getBoosterVersion(data): #Definimos la función
    for x in data['rocket']: #Iteramos por cada fila de la columna rocket
       if x: #Comprobamos que no este vacía
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
        response = response.json() #Transformamos la respuesta en un diccionario.
        BoosterVersion.append(response['name']) #Añadimos a BoosterVersion el valor con key 'name'

De <code>launchpad</code> nos gustaría quedarnos con la longitud, la latitud y el nombre de la plataforma


In [76]:
# DEFINICIÓN FUNCIÓN 2
#INSERTA AQUI TU CÓDIGO

def getLaunchSite(data): #Definimos la función getLaunchSite que recibe con argumento data
   for x in data['launchpad']: #Iteramos por cada fila de la columna launchpad
    if x:#Comprobamos que no este vacía
      response = requests.get("https://api.spacexdata.com/v4/launchpads/" + str(x)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
      response = response.json() #Transformamos la respuesta en un diccionario.
      Longitude.append(response['longitude']) #Añadimos a Longitude el valor con key 'longitude'
      Latitude.append(response['latitude']) #Añadimos a Latitude el valor con key 'latitude'
      LaunchSite.append(response['name']) #Añadimos a LaunchSite el valor con key 'name'

De <code>payload</code> nos gustaría obtener la carga, el peso de la carga, el cliente y la órbita a la que fué enviada.

In [77]:
# DEFINICIÓN FUNCIÓN 3
#INSERTA AQUI TU CÓDIGO


def getPayloadData(data): #Definimos la función getPayloadData
  for x in data['payloads']: #Iteramos por cada fila de la columna payloads
    if x: #Comprobamos que no este vacía
      response = requests.get("https://api.spacexdata.com/v4/payloads/" + str(x)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
      response = response.json() #Transformamos la respuesta en un diccionario.
      Payload.append(response['name']) #Añadimos a Payload el valor con key 'name'
      Customers.append(response['customers']) #Añadimos a Customers el valor con key 'customers'
      PayloadMass.append(response['mass_kg']) #Añadimos a PayloadMass el valor con key 'mass_kg'
      Orbit.append(response['orbit']) #Añadimos a Orbit el valor con key 'orbit'

La función para completar los datos de `core` tiene una complicación extra por lo que la damos hecha:

In [78]:
# DEFINICIÓN FUNCIÓN 4
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])


Las funciones anteriores rellenan las listas vacías, veamoslo en práctica. *Revisa que la lista BoosterVersion esta vacía*


In [79]:
#INSERTA AQUI TU CÓDIGO
BoosterVersion

[]

*Llama ahora la función `getBoosterVersion`.* (No devuelve nada así que no hace falta asignarle una variable. Solo instanciamos la funcion.)

In [80]:
# LLAMADA A FUNCIÓN 1
#INSERTA AQUI TU CÓDIGO
getBoosterVersion(data)

*Muestra los primeros 5 valores de la lista `BoosterVersion`*

In [81]:
#INSERTA AQUI TU CÓDIGO
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

*Aplica el resto de funciones:*
(No devuelven nada así que no hace falta asignarles una variable. Solo instanciamos a las funciones.)

In [82]:
# LLAMADA A FUNCIÓN 2
#INSERTA AQUI TU CÓDIGO
getPayloadData(data)

In [83]:
# LLAMADA A FUNCIÓN 3
#INSERTA AQUI TU CÓDIGO
getLaunchSite(data)

In [84]:
# LLAMADA A FUNCIÓN 4
#INSERTA AQUI TU CÓDIGO
getCoreData(data)

Finalmente creamos un dataframe desde las listas anteriores. Primero convirtámolos en un diccionario:

In [85]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Payload':Payload,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'LandingOutcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'Customers': Customers,
'Mission_Outcome':list(data['success']),
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

Y ahora crea un dataframe de Pandas desde el diccionario. Llámalo `launch_data`

In [86]:
#INSERTA AQUI TU CÓDIGO
launch_data = pd.DataFrame(launch_dict)

Finalmente, muestra las primeras 5 filas del dataframe.

In [87]:
#INSERTA AQUI TU CÓDIGO
launch_data.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,FalconSAT-2,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,DemoSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,RatSat,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[SpaceX],True,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,RazakSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[ATSB],True,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857


### Parte 2: Filtra el dataframe para que solo incluya los lanzamientos de falcon 9



El falcon 9 es una versión considerablemente más nueva y efectiva que el faclon 1. Dado que nos interesan los datos recientes y a futuro vamos a *eliminar los lanzamientos de `Falcon 1` del fataframe launch_data y llama al nuevo dataframe `data_falcon9`. Muestra las primeras 5 filas.*

In [88]:
# INSERTA AQUÍ TU CÓDIGO
data_falcon9 = launch_data.loc[launch_data['BoosterVersion']== "Falcon 9", :] 
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.0,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA(COTS)],True,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.0,CRS-2,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA (CRS)],True,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.0,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,[MDA],True,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.0,SES-8,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SES],True,0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,181,2022-08-28,Falcon 9,13260.0,Starlink 4-23 (v1.5),VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3033383ecb075134e7cd,5.0,[SpaceX],True,1,B1069,-80.603956,28.608058
168,182,2022-08-31,Falcon 9,13260.0,Starlink 3-4 (v1.5),VLEO,VAFB SLC 4E,True ASDS,7,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,[SpaceX],True,6,B1063,-120.610829,34.632093
169,185,2022-09-17,Falcon 9,13260.0,Starlink 4-34 (v1.5),VLEO,CCSFS SLC 40,True ASDS,6,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,[SpaceX],True,5,B1067,-80.577366,28.561857
170,186,2022-09-24,Falcon 9,13260.0,Starlink 4-35 (v1.5),VLEO,CCSFS SLC 40,True ASDS,4,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,[SpaceX],True,0,B1072,-80.577366,28.561857


Ahora que hemos eliminado los lanzamientos de Falcon 1 la columna FlightNumber está desajustada. Vamos a volver a ordenarla:

In [90]:
data_falcon9.FlightNumber = list(range(1, data_falcon9.shape[0]+1))
data_falcon9.head()

<ipython-input-90-5e3e7c2c50f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9.FlightNumber = list(range(1, data_falcon9.shape[0]+1))


,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA(COTS)],True,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,CRS-2,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA (CRS)],True,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,[MDA],True,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,SES-8,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SES],True,0,B1004,-80.577366,28.561857


### Parte 3: Ajustes finales


Debajo podemos observar que nos faltan ciertos valores.

In [91]:
data_falcon9.isnull().sum()

FlightNumber        0
Date                0
BoosterVersion      0
PayloadMass        22
Payload             0
Orbit               1
LaunchSite          0
LandingOutcome      0
Flights             0
GridFins            0
Reused              0
Legs                0
LandingPad         26
Block               0
Customers           0
Mission_Outcome     1
ReusedCount         0
Serial              0
Longitude           0
Latitude            0
dtype: int64

* Lo óptimo sería no tener celdas vacías por lo que vamos a ver que podemos hacer para solucionar esta situación.
La columna <code>LandingPad</code> tiene valores `None`en los casos donde el cohete no tuvo intención de aterrizar por lo que no tenía ningún LandingPad asignado. 

* Por otro lado, los valores innexistentes de `PayloadMass` si se pueden completar. *En este caso utilizaras la función `.mean()` para calcular la media y `.replace()` para reemplzar los valores `np.nan`.* 


In [92]:
# INSERTA AQUI TU CÓDIGO
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].replace(np.nan, data_falcon9['PayloadMass'].mean())

data_falcon9.isnull().sum()

<ipython-input-92-cdb74e1d606a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].replace(np.nan, data_falcon9['PayloadMass'].mean())


FlightNumber        0
Date                0
BoosterVersion      0
PayloadMass         0
Payload             0
Orbit               1
LaunchSite          0
LandingOutcome      0
Flights             0
GridFins            0
Reused              0
Legs                0
LandingPad         26
Block               0
Customers           0
Mission_Outcome     1
ReusedCount         0
Serial              0
Longitude           0
Latitude            0
dtype: int64

El número de celdas vacías en la columna <code>PayLoadMass</code> debería cambiar a 0.


Por último tenemos que la columna `Customers` es una lista, y como hemos comentado previamente, no se llevan especialmente bien con SQL. 
*Transforma esta lista en una string aplicando `.map(lambda x : ','.join(x))` a la columna y guardalo en la misma columna.*

In [93]:
#INSERTA AQUÍ TU CÓDIGO
data_falcon9['Customers'] = data_falcon9['Customers'].map(lambda x : ','.join(x))

data_falcon9

<ipython-input-93-cf6a2f4afbe1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9['Customers'] = data_falcon9['Customers'].map(lambda x : ','.join(x))


,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,8191.07911,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,SpaceX,True,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.00000,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,NASA(COTS),True,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.00000,CRS-2,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,NASA (CRS),True,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.00000,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,MDA,True,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.00000,SES-8,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,SES,True,0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,164,2022-08-28,Falcon 9,13260.00000,Starlink 4-23 (v1.5),VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3033383ecb075134e7cd,5.0,SpaceX,True,1,B1069,-80.603956,28.608058
168,165,2022-08-31,Falcon 9,13260.00000,Starlink 3-4 (v1.5),VLEO,VAFB SLC 4E,True ASDS,7,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,SpaceX,True,6,B1063,-120.610829,34.632093
169,166,2022-09-17,Falcon 9,13260.00000,Starlink 4-34 (v1.5),VLEO,CCSFS SLC 40,True ASDS,6,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,SpaceX,True,5,B1067,-80.577366,28.561857
170,167,2022-09-24,Falcon 9,13260.00000,Starlink 4-35 (v1.5),VLEO,CCSFS SLC 40,True ASDS,4,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,SpaceX,True,0,B1072,-80.577366,28.561857


*Ahora exporta el dataset en formato `.csv`  con `index=False` y llámalo `dataset_part_1.csv`. Descarga el archivo porque será necesario en la siguiente práctica.*


In [94]:
#INSERTA AQUÍ TU CÓDIGO
data_falcon9.to_csv('dataset_part_1.csv', index=False)
